In [101]:
import pandas as pd
import nltk
import string
import numpy as np

In [8]:
articles = []

for l in range(1,4):
    k = 32
    if (l == 2):
        k=30     
    for i in range(1,k):
        dataP = pd.read_csv(str(l) + "_" + str(i) + "_P.csv")
        dataTH = pd.read_csv(str(l) + "_" + str(i) + "_TH.csv")
        dataTOI = pd.read_csv(str(l) + "_" + str(i) + "_TOI.csv")
        for j in range(len(dataP)):
            articles.append(dataP['Article'][j])
        for j in range(len(dataTH)):
            articles.append(dataTH['Content'][j])
        for j in range(len(dataTOI)):
            articles.append(dataTOI['Content'][j])
                     
     

In [9]:
print(len(articles))

105006


In [11]:
for i in range(len(articles)):
    if type(articles[i]) is not str:
        del articles[i]
    articles[i] = articles[i].lower()
   
    

In [12]:
for i in range(len(articles)):
    articles[i] = articles[i].translate(str.maketrans('','' , string.punctuation))

In [13]:
for i in range(len(articles)):
    articles[i] = " ".join(articles[i].split())

In [14]:
from nltk import word_tokenize
words = []


In [15]:
for i in range(len(articles)):
    words.append(word_tokenize(articles[i]))

In [16]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))


In [17]:
for i in range(len(words)):
    words[i] = [word for word in words[i] if word not in stop_words]  

In [18]:
for i in range(len(words)):
    words[i] = [token for token in words[i] if not token.isnumeric()] 

In [19]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
  

for i in range(len(words)):
    words[i] = [lemmatizer.lemmatize(j) for j in words[i]]

In [20]:
for i in range(len(words)):    
    words[i] = [token for token in words[i] if len(token) > 1]

In [21]:
from gensim.models import Phrases

In [22]:
bigram = Phrases(words, min_count=5,threshold= 100)

In [23]:
for idx in range(len(words)):
    for token in bigram[words[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            words[idx].append(token)    



In [24]:
from gensim.corpora import Dictionary

In [45]:
dictionary = Dictionary(words)
dictionary.filter_extremes(no_below =5000 ,no_above=0.3, keep_n = 100000)

In [46]:
corpus = [dictionary.doc2bow(doc) for doc in words]


In [47]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [129]:
from gensim.models import LdaModel
num_topics = 7
chunksize = 5000
passes = 20
iterations = 400
eval_every = None

In [130]:
temp = dictionary[0]

In [131]:
id2word = dictionary.id2token

In [132]:
model = LdaModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [ ]:
top_topics = model.top_topics(corpus)
for t in top_topics:
    print(t)
    print("\n")

In [59]:
import pyLDAvis.gensim

In [133]:
lda_display = pyLDAvis.gensim.prepare(model, corpus_tfidf, dictionary, sort_topics=False)


C:\Users\ajain\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [134]:
pyLDAvis.display(lda_display)


In [62]:
all_topics = model.get_document_topics(corpus_tfidf, per_word_topics=False)


In [ ]:
for i in range(0,10):
    print('New Document \n')
    print('Document topics:', all_topics[i])
    print(" ")
    print('-------------- \n')

In [ ]:
print( type(all_topics[0]))

In [ ]:
print(articles[0])


In [ ]:
import numpy as np

doc1 = np.array([tup[1] for tup in all_topics[166]])
doc2 = np.array([tup[1] for tup in all_topics[205]])

In [ ]:
print(doc1)
print(doc2)


In [ ]:
from scipy.spatial import distance

In [ ]:
distance.jensenshannon(doc1,doc2)

In [ ]:
print(articles[2])

In [ ]:
print(articles[9])

In [140]:
from gensim.test.utils import datapath
    
temp_file = datapath("model_mar")
model.save(temp_file)

model_mar = LdaModel.load(temp_file)

In [ ]:
print(temp_file)

In [126]:
from gensim.models import hdpmodel
hdp_model = hdpmodel.HdpModel(corpus=corpus_tfidf, id2word=id2word)
hdp_model.print_topics(num_topics=-1)

[]

In [127]:
    shown_topics = hdp_model.show_topics(num_topics=hdp_model.m_T, formatted=False)
    topics_nos = [x[0] for x in shown_topics ]
    weights = [ sum([item[1] for item in shown_topics[topicN][1]]) for topicN in topics_nos ]
    ll= pd.DataFrame({'topic_id' : topics_nos, 'weight' : weights})

In [128]:
for i in range(0,150):
    print(ll['weight'][i])
    print('\n')

0.16216205203012626


0.14772915453611407


0.15047052967935481


0.16413260141073824


0.15749940944500077


0.18398068901400363


0.15119530008018536


0.1578810119091664


0.15028571947835193


0.15529566660956487


0.1513781562973909


0.17044351220329124


0.15581832338752458


0.15777575441686315


0.15762238918912355


0.15859322440296397


0.1684186079280834


0.16780715213519007


0.15261118389731976


0.15127353218627562


0.14976108176258116


0.16161461528803917


0.15835301552310524


0.17249495704542078


0.15236454566507734


0.16782478219517127


0.15888184038817335


0.15887484965417523


0.18291026277661834


0.17242954436584373


0.15548852349212477


0.1466431602382019


0.15293137446477342


0.16365983815360594


0.1555508133532584


0.15890354094513096


0.16756760914830968


0.16230737533559034


0.15506698752226214


0.15136383631985265


0.17260226930267233


0.15468454729711334


0.16731914090704098


0.15853912141057688


0.17723614362651702


0.1623087312597

In [137]:
train_vecs = []
for i in range(0,len(words)):
    top_topics = model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(7)]
    #topic_vec.extend([words.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(words.iloc[i].text)]) # length review
    train_vecs.append(topic_vec)

In [138]:
print(train_vecs[0])

[0.8471955, 5.7937516e-05, 6.454417e-05, 0.15250175, 6.404283e-05, 6.623615e-05, 4.9979626e-05]


In [125]:
X = np.array(train_vecs)
y = np.array(words)
from sklearn.preprocessing import MultiLabelBinarizer



from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    print(type(y_train))
    mlb = MultiLabelBinarizer(sparse_output=True)
    mlb.fit_transform(y_train)
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)
    
    
    
    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

<class 'numpy.ndarray'>


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.